In [193]:
import numpy as np
import pandas as pd

#### Size system components e.g. capacitor given the amount of energy needed to be stored. 

#### Find the maximum and minimum net energy available given different COD and storage capacitors. 

In [219]:
# Energy harvested from single batch feed MFC, Joules (J) for different COD values

Eave = np.array([21.137,  # COD = 900
                 18.017,  # COD = 500
                 8.315,  # COD = 300
                 2.415    # COD = 70
                ])

Eave *= 0.5 # apply energy harvesting eff.
Eave

array([10.5685,  9.0085,  4.1575,  1.2075])

Maximum and minimum rated supply voltages (e.g. for regulator to supply 1.8V) for power management components.

Two system configurations (different combinations of components considerd) 

In [220]:
# max allowable voltage for comparator and voltage regulator, V for two configs : a & b
# a : comp:LTC1540, reg:LT3009
# b : comp:MAX9117, reg:TPS7A02

Vmax_a = 11
Vmax_b = 6

Vmin_a = 2.3
Vmin_b = 2.3

Two system configurations (different combinations of components considerd) 

In [221]:
# Create a data frame for each system config and each COD value

sys_a = pd.DataFrame(index=['cod_900', 'cod_500', 'cod_300', 'cod_70'])
sys_b = pd.DataFrame(index=['cod_900', 'cod_500', 'cod_300', 'cod_70'])

sys_a['Eave'] = Eave
sys_b['Eave'] = Eave

Capacitor energy $E = \frac{1}{2} CV^2$

Capacitance needed to store Emax (average energy harvetsed at each COD value) at Vmax found by:

$C = \frac{E_{max}}{\frac{1}{2} V_{max}^2}$

In [222]:
# capacitor needed to store Emax for each COD value at Vmax for each system config

sys_a.insert(0, 'C', sys_a.Eave / (0.5 * Vmax_a**2))
sys_b.insert(0, 'C', sys_b.Eave / (0.5 * Vmax_b**2))


# display(sys_a) 
# display(sys_b)

For a capacitor of this size, the remaining energy $E_{min}$ stored in the capacitor when drained to the minimum allowable supply voltage for the system $V_{min}$ found by: 

$E_{min} = \frac{1}{2} CV_{min}^2$

In [223]:
# Average energy harvested at lowest COD value tested (COD = 70, 900)

sys_a['Ecod_70'] = Eave[-1]
sys_b['Ecod_70'] = Eave[-1]

sys_a['Ecod_900'] = None
sys_b['Ecod_900'] = None

sys_a.loc['cod_900', 'Ecod_900'] = Eave[0]
sys_b.loc['cod_900', 'Ecod_900'] = Eave[0]

# display(sys_a) 
# display(sys_b)

In [224]:
# Energy remaining at whicever is lower out of :
# a) the cut-off voltage of each system config 
# b) the voltage the capacitor is charged to using at average energy harvested during one batch at COD=70
# (Energy at Vmin)

sys_a['Emin'] = min(0.5 * sys_a.C * Vmin_a**2)#, sys_a['Ecod_70'])
sys_b['Emin'] = min(0.5 * sys_b.C * Vmin_b**2)#, sys_a['Ecod_70'])

In [225]:
s = pd.Series(sys_a.loc[:, ['Ecod_70', 'Emin']].min(axis=1))
s = pd.Series(sys_b.loc[:, ['Ecod_70', 'Emin']].min(axis=1))

The energy $E_{a}$ availabe for use, COD 900

$E_{a} = E_{max,COD=900} - E_{min}$

In [226]:
# # Max net energy available for use
sys_a['Ea_cod_900'] = sys_a.Ecod_900 - sys_a.Emin
sys_b['Ea_cod_900'] = sys_b.Ecod_900 - sys_b.Emin

# display(sys_a) 
# display(sys_b)

The energy $E_{a}$ availabe for use, COD 70

$E_{a} = E_{max,COD=70} - E_{min}$

In [227]:
# # Min net energy available for use
sys_a['Ea_cod_70'] = sys_a.Ecod_70 - sys_a.Emin
sys_b['Ea_cod_70'] = sys_b.Ecod_70 - sys_b.Emin

# display(sys_a) 
# display(sys_b)

#### $E_a$ determines how many samples can be taken by the data logger in the next batch. 

#### Use the energy consumption of each process done by the microcontroller and power management system to calculate the allowable number of samples corresponding to each COD value 

In [228]:
Vop = 1.8 # + voltage pic microcontroller


# Energy consumed per ML event instance for each event type 

Eout = pd.DataFrame({ 'Current':  [1.18e-3, 2.85e-3, 2e-8], # A
                      'ExecTime':[0.1, 0.3, 7*24*60*60]}, # s
                    index=['sampling', 'estimation', 'sleep'])

Eout

,Current,ExecTime
sampling,1.180000e-03,0.1
estimation,2.850000e-03,0.3
sleep,2.000000e-08,604800.0


In [229]:
# energy consumed by power mgmt system a
# a : comp:LTC1540, reg:LT3009
Eout_a = pd.DataFrame({'Current':  [0.3e-6, 3e-6], # A
                       'ExecTime': [7*24*60*60, 7*24*60*60]}, # s
                     index=        ['comparator', 'regulator'])


# energy consumed by power mgmt system b
# b : comp:MAX9117, reg:TPS7A02
Eout_b = Eout_a
Eout_b.loc['comparator','Current'] = 0.6e-6
Eout_b.loc['regulator','Current'] = 25e-9

In [230]:
# Data frame for each system
Eout_a = Eout.append(Eout_a)
Eout_b = Eout.append(Eout_b)

Current power execution time and energy for each energy-consuming process

In [231]:
Eout_a['Power'] = Eout_a.Current  * Vop # W
Eout_b['Power'] = Eout_b.Current  * Vop # W

Eout_a['Energy'] = Eout_a.Power * Eout_a.ExecTime # J
Eout_b['Energy'] = Eout_b.Power * Eout_b.ExecTime # J

In [232]:
Eout_a

,Current,ExecTime,Power,Energy
sampling,1.180000e-03,0.1,2.124000e-03,0.000212
estimation,2.850000e-03,0.3,5.130000e-03,0.001539
sleep,2.000000e-08,604800.0,3.600000e-08,0.021773
comparator,6.000000e-07,604800.0,1.080000e-06,0.653184
regulator,2.500000e-08,604800.0,4.500000e-08,0.027216


In [233]:
def n_samples(Ea, Eout):
    return ( Ea - 
             Eout.Energy['estimation'] - 
             Eout.Energy['sleep'] -
             Eout.Energy['comparator'] -
             Eout.Energy['regulator']       
           ) / Eout.Energy['sampling'] 

Maximum allowable number of samples (COD = 900) 

In [234]:
# # Max samples
sys_a['Nsamp_max_cod_900'] = n_samples(sys_a.Ea_cod_900, Eout_a)
sys_b['Nsamp_max_cod_900'] = n_samples(sys_b.Ea_cod_900, Eout_b)


# display(sys_a) 
# display(sys_b)

Minumum time interval betweeen samples (COD = 900) 

In [235]:
# Sampling time interval
sys_a['Nsamp_min_int_cod_900'] = 7*24*60*60 / sys_a.Nsamp_max_cod_900
sys_b['Nsamp_min_int_cod_900'] = 7*24*60*60 / sys_b.Nsamp_max_cod_900


# display(sys_a) 
# display(sys_b)

Maximum allowable number of samples (COD = 70) 

In [236]:
# Min samples
sys_a['Nsamp_max_cod_70'] = n_samples(sys_a.Ea_cod_70, Eout_a)
sys_b['Nsamp_max_cod_70'] = n_samples(sys_b.Ea_cod_70, Eout_b)


#display(sys_a) 
#display(sys_b)

Minumum time interval betweeen samples (COD = 70) 

In [237]:
# Sampling time interval
sys_a['Nsamp_min_int_cod_70'] = 7*24*60*60 / sys_a.Nsamp_max_cod_70
sys_b['Nsamp_min_int_cod_70'] = 7*24*60*60 / sys_b.Nsamp_max_cod_70


display(sys_a) 
display(sys_b)

,C,Eave,Ecod_70,Ecod_900,Emin,Ea_cod_900,Ea_cod_70,Nsamp_max_cod_900,Nsamp_min_int_cod_900,Nsamp_max_cod_70,Nsamp_min_int_cod_70
cod_900,0.174686,10.5685,1.2075,10.5685,0.052791,10.5157,1.154709,46195.8,13.0921,2123.340384,284.834219
cod_500,0.148901,9.0085,1.2075,None,0.052791,NaN,1.154709,NaN,NaN,2123.340384,284.834219
cod_300,0.068719,4.1575,1.2075,None,0.052791,NaN,1.154709,NaN,NaN,2123.340384,284.834219
cod_70,0.019959,1.2075,1.2075,None,0.052791,NaN,1.154709,NaN,NaN,2123.340384,284.834219


,C,Eave,Ecod_70,Ecod_900,Emin,Ea_cod_900,Ea_cod_70,Nsamp_max_cod_900,Nsamp_min_int_cod_900,Nsamp_max_cod_70,Nsamp_min_int_cod_70
cod_900,0.587139,10.5685,1.2075,10.5685,0.177435,10.3911,1.030065,45609,13.2605,1536.500863,393.621647
cod_500,0.500472,9.0085,1.2075,None,0.177435,NaN,1.030065,NaN,NaN,1536.500863,393.621647
cod_300,0.230972,4.1575,1.2075,None,0.177435,NaN,1.030065,NaN,NaN,1536.500863,393.621647
cod_70,0.067083,1.2075,1.2075,None,0.177435,NaN,1.030065,NaN,NaN,1536.500863,393.621647


When COD is only 70, the mimimum allowable sampling time is approx 5mins. 

The following calculations for the comparator threshold voltages Vth, Vtl are based on the principle that the minimum energy for the system to switch on (Vth) should be the average energy harvested Eave when COD=70. 

The reason for this is that the average energy Eave when COD=70 should provide enough energy to sample and analyse a subsequent peak (COD $\geqslant$ 70), at the calculated sampling rate. 

If the sample switches on at a lower energy level, it is in danger of running out of energy while completing the sampling task. 

(NB: If a lower samplign rate is selected, a lower Vth can be calculated) 

<br>





In [240]:
# Upper threshold voltage for comparator  
sys_a['Vth,h'] = np.sqrt( 2 * sys_a.Ecod_70 / sys_a.C )
sys_b['Vth,h'] = np.sqrt( 2 * sys_b.Ecod_70 / sys_b.C )

# display(sys_a) 
# display(sys_b)

The lower threshold Vtl is determined by the lowest allowable input volatge of the power management sytsem, defined earlier. 

In [242]:
# Lower threshold voltage for comparator  
sys_a['Vth,l'] = Vmin_a
sys_b['Vth,l'] = Vmin_b

print('System a')
display(sys_a) 
print('System b')
display(sys_b)

System a


,C,Eave,Ecod_70,Ecod_900,Emin,Ea_cod_900,Ea_cod_70,Nsamp_max_cod_900,Nsamp_min_int_cod_900,Nsamp_max_cod_70,Nsamp_min_int_cod_70,"Vth,h","Vth,l"
cod_900,0.174686,10.5685,1.2075,10.5685,0.052791,10.5157,1.154709,46195.8,13.0921,2123.340384,284.834219,3.718173,2.3
cod_500,0.148901,9.0085,1.2075,None,0.052791,NaN,1.154709,NaN,NaN,2123.340384,284.834219,4.027263,2.3
cod_300,0.068719,4.1575,1.2075,None,0.052791,NaN,1.154709,NaN,NaN,2123.340384,284.834219,5.928163,2.3
cod_70,0.019959,1.2075,1.2075,None,0.052791,NaN,1.154709,NaN,NaN,2123.340384,284.834219,11.000000,2.3


System b


,C,Eave,Ecod_70,Ecod_900,Emin,Ea_cod_900,Ea_cod_70,Nsamp_max_cod_900,Nsamp_min_int_cod_900,Nsamp_max_cod_70,Nsamp_min_int_cod_70,"Vth,h","Vth,l"
cod_900,0.587139,10.5685,1.2075,10.5685,0.177435,10.3911,1.030065,45609,13.2605,1536.500863,393.621647,2.028094,2.3
cod_500,0.500472,9.0085,1.2075,None,0.177435,NaN,1.030065,NaN,NaN,1536.500863,393.621647,2.196689,2.3
cod_300,0.230972,4.1575,1.2075,None,0.177435,NaN,1.030065,NaN,NaN,1536.500863,393.621647,3.233543,2.3
cod_70,0.067083,1.2075,1.2075,None,0.177435,NaN,1.030065,NaN,NaN,1536.500863,393.621647,6.000000,2.3


System B : the voltage of the capacitor when charged using the average energy harvested Eave when COD=70 < minimum supply voltage for the electronics. 
 
This shows that system B is not suitable.

Use system A and calculate comparator hysteresis based on these values. 